Bokeh test

In [ ]:
import numpy as np
import scanpy as sc
import os
import requests
try:
  import xomx
except ImportError:
    !pip install git+https://github.com/perrin-isir/xomx
import xomx
import bokeh
import bokeh.plotting
import bokeh.io
bokeh.io.output_notebook()

rg = np.random.default_rng()
x = np.linspace(-3, 3, 5000)
y = rg.normal(x, 1)

In [ ]:
p = bokeh.plotting.figure(
    x_axis_label='x',
    y_axis_label='y',
    tools="hover, pan, wheel_zoom, box_zoom, reset",
    tooltips = [('annotation', '@annotation')]
)

In [ ]:
data=dict(
    x=x,
    y=y,
    # colors=color,
    annotation=np.arange(5000),
)
p.scatter('x', 'y', 4, 4, source=data)
# p.circle(x,y)
bokeh.io.show(p)

In [ ]:
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')
np.random.seed(37)
violin = hv.Violin((np.random.randn(2000)), vdims='Value')
violin.opts(violin_width=0.2, inner=None)
# quartiles = violin.opts(opts.Violin(inner='quartiles', cut=1.))
quart = hv.render(violin)
bokeh.io.show(quart)

In [ ]:
from bokeh.models import HoverTool
from bokeh.sampledata.periodic_table import elements
from holoviews import dim, opts

points = hv.Points(
    elements, ['electronegativity', 'density'],
    ['name', 'symbol', 'metal', 'CPK', 'atomic radius']
).sort('metal')

tooltips = [
    ('Name', '@name'),
    ('Symbol', '@symbol'),
    ('CPK', '$color[hex, swatch]:CPK')
]
hover = HoverTool(tooltips=tooltips)

points.opts(
    tools=[hover], color='atomic radius', cmap='Category20',
    line_color='black', size=dim('atomic radius')/10,
    width=900, height=600, show_grid=False,
    title='Chemical Elements by Type (scaled by atomic radius)')

In [ ]:
elements

In [ ]:
# To display interactive plots:
%matplotlib widget
save_dir = os.path.join(os.path.expanduser('~'), 'results', 'xomx', 'tutorials', 'xomx_pbmc')
os.makedirs(save_dir, exist_ok=True)
# Setting the pseudo-random number generator
rng = np.random.RandomState(0)
pbmc3k_file = 'pbmc3k.tar.gz'
if not os.path.isfile(os.path.join(save_dir, pbmc3k_file)):
    url = (
        "https://cf.10xgenomics.com/samples/cell/pbmc3k/"
        + "pbmc3k_filtered_gene_bc_matrices.tar.gz"
    )
    r = requests.get(url, allow_redirects=True)
    open(os.path.join(save_dir, "pbmc3k.tar.gz"), "wb").write(r.content)
    os.popen(
        "tar -xzf " + os.path.join(save_dir, "pbmc3k.tar.gz") + " -C " + save_dir
    ).read()
xd = sc.read_10x_mtx(
os.path.join(save_dir, "filtered_gene_bc_matrices", "hg19"),
var_names="gene_symbols",
)
xd.var_names_make_unique()
sc.pp.filter_cells(xd, min_genes=200)
sc.pp.filter_genes(xd, min_cells=3)
xd.var["mt"] = xd.var_names.str.startswith(
    "MT-"
)  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(
    xd, qc_vars=["mt"], percent_top=None, log1p=False, inplace=True
)
# The k-th element of the following array is the mean fraction of counts of the
# k-th gene in each single cell, across all cells
mean_count_fractions = np.squeeze(
    np.asarray(
        np.mean(
            xd.X / np.array(xd.obs["total_counts"]).reshape((xd.n_obs, 1)), axis=0
        )
    )
)


In [ ]:
# Plot, for all genes, the mean fraction
# of counts in single cells, across all cells
pts = xomx.pl.function_plot(
    xd,
    lambda idx: mean_count_fractions[idx],
    obs_or_var="var",
    violinplot=False,
    ylog_scale=True,
    xlabel="genes",
    ylabel="mean fractions of ..",
)

In [ ]:
xd.uns['var_indices'] = xomx.tl.var_indices(xd)
xomx.pl.var_plot(xd, "LINC00115")